<a href="https://colab.research.google.com/github/pedropalmav/proyecto-recsys/blob/main/MistralLora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load Dataset

In [1]:
!wget -O LLM-ReDial-2024.zip "https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1"
!unzip LLM-ReDial-2024.zip
!unzip LLM_Redial.zip
!rm LLM_Redial.zip
!rm LLM-ReDial-2024.zip
!rm -rf __MACOSX

--2024-12-09 15:25:35--  https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd6c90b0cfcb8ca9cfb909533da.dl.dropboxusercontent.com/cd/0/inline/Cf4DZS33Q7BLTqSfncZGTYNgjaZ2SNSJcvLFMqC1FdVJDhyf6IqwGd6fpcuErVbbdxLZFLCV3mDHCiwf83DpfyETOxB7CLgCr2bSlotzD21rG5R-ATH30IWIvCjWESnDcws/file?dl=1# [following]
--2024-12-09 15:25:36--  https://ucd6c90b0cfcb8ca9cfb909533da.dl.dropboxusercontent.com/cd/0/inline/Cf4DZS33Q7BLTqSfncZGTYNgjaZ2SNSJcvLFMqC1FdVJDhyf6IqwGd6fpcuErVbbdxLZFLCV3mDHCiwf83DpfyETOxB7CLgCr2bSlotzD21rG5R-ATH30IWIvCjWESnDcws/file?dl=1
Resolving ucd6c90b0cfcb8ca9cfb909533da.dl.dropboxusercontent.com (ucd6c90b0cfcb8ca9cfb909533da.dl.dropboxusercontent.com)... 162.125.65.15, 2620:100:6021:15::

#Load HF model

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!pip install transformers[torch]
!pip install bitsandbytes
!pip install peft
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [7]:
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [8]:
def generate_prompt(data):
  question = data['question']
  ref = data['references']

  chat = [
      {"role": "user", "content": f"Answer the Question: {question} Considering this information: {ref}"},
    ]

  data['prompt'] = tokenizer.apply_chat_template(chat, tokenize=False)
  return data

#dataset['test'] = dataset['test'].map(generate_prompt)

In [9]:
def generate_zeroshot_prompt(likes, dislikes):
  likes_str = "\n".join(likes)
  dislikes_str = "\n".join(dislikes)

  chat = [
    {"role": "user", "content": f"I've liked these movies: {likes_str}. I've disliked these movies: {dislikes_str}. Recommend a new movie for me to watch."}
  ]
  return tokenizer.apply_chat_template(chat, tokenize=False)

In [10]:
def generate_zeroshot(conversation_id, conversations):
    conversation = "\n".join(conversations[conversation_id]["messages"])
    content = (
        f"Pretend you are a movie recommender system. I will give you a conversation between a human and assistant. Based on the conversation, you reply me with 10 recommendations, including only their titles without extra sentences or description. Here is the conversation: {conversation}.\n"
        f"\n"
    )
    messages = [
        {"role": "user", "content": content}
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    return prompt

In [11]:
from Tools import read_user_data
path_movie_data = "/content/Movie/final_data.jsonl"

In [12]:
import json
item_map_path = "/content/Movie/item_map.json"
item_map = json.load(open(item_map_path,'r'))
user_map_path = "/content/Movie/user_ids.json"
user_map = json.load(open(user_map_path,'r'))

In [13]:
conversations_path = "/content/Movie/Conversation.txt"

conversations = []
current_conversation = []
conversation_id = 0
with open(conversations_path, 'r') as file:
    for line in file:
        line = line.strip()
        if not line:
            continue

        if line.isdigit():
            if current_conversation:
                conversations.append({
                    'conversation_id': conversation_id,
                    'messages': current_conversation
                })
            conversation_id = int(line)
            current_conversation = []
        else:
            current_conversation.append(line)

    if current_conversation:
        conversations.append({
            'conversation_id': conversation_id,
            'messages': current_conversation
        })

## DATASET

In [14]:
import random
user_ids = list(user_map.keys())
sample_percent = 0.1

sample_size = int(sample_percent * len(user_ids))
sampled_users = random.sample(user_ids,sample_size)

In [15]:
dataset = []
for user_id in sampled_users:
  user_data = read_user_data(path_movie_data, user_id)
  for conv_number, conversation in enumerate(user_data['Conversation']):
    conversation_id = conversation[f"conversation_{conv_number+1}"]["conversation_id"]
    prompt = generate_zeroshot(conversation_id, conversations)
    dataset.append({
        "user_id": user_id,
        "conversation_id": conversation_id,
        "user_might_like": user_data['user_might_like'],
        "prompt": prompt
    })

In [16]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)
print("Train prompts:", len(train_set))
print("Test prompts:", len(test_set))

Train prompts: 782
Test prompts: 196


In [24]:
conversations_id = []
conversations_train = []
for data in train_set:
  conversations_id.append(data['conversation_id'])

for conversation in conversations:
  if conversation['conversation_id'] in conversations_id:
    conversations_train.append(conversation)

## Lora

In [25]:
import os
import peft

#os.environ["CUDA_VISIBLE_DEVICES"] = "0" # si solo vamos a usar la primera GPU que tengamos

import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, AutoConfig, GPTJForCausalLM
from peft import LoraConfig, get_peft_model

In [26]:
# Settings for A100 - For 3090
MICRO_BATCH_SIZE = 4  # change to 8 for A100
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 2  # paper uses 3
LEARNING_RATE = 2e-5
CUTOFF_LEN = 256
LORA_R = 4
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

In [27]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

In [28]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

dataset = Dataset.from_list(conversations_train)

def flatten_and_tokenize_function(examples):
    combined_texts = []
    for messages in examples['messages']:
        flattened_messages = []
        for message in messages:
            if isinstance(message, list):
                flattened_messages.extend(message)
            else:
                flattened_messages.append(message)
        combined_texts.append(" ".join(flattened_messages))

    return tokenizer(
        combined_texts,
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_dataset = dataset.map(flatten_and_tokenize_function, batched=True, remove_columns=["conversation_id", "messages"])

training_args = TrainingArguments(
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    warmup_steps=100,
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=1,
    output_dir="./result",
    save_total_limit=3,
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train(resume_from_checkpoint=False)

Map:   0%|          | 0/782 [00:00<?, ? examples/s]

Step,Training Loss
1,2.575500
2,2.571000
3,2.536200
4,2.564900
5,2.587800
6,2.542700
7,2.567600
8,2.543000
9,2.496000
10,2.603100


TrainOutput(global_step=12, training_loss=2.5638017654418945, metrics={'train_runtime': 292.9971, 'train_samples_per_second': 5.338, 'train_steps_per_second': 0.041, 'total_flos': 3322151280574464.0, 'train_loss': 2.5638017654418945, 'epoch': 1.9591836734693877})

In [33]:
trainer.save_model("Llama-3.2-3B-Instruct.h5")

In [34]:
#mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [35]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

base_model = "meta-llama/Llama-3.2-3B-Instruct"
adapter_model = "Llama-3.2-3B-Instruct.h5"

model = AutoModelForCausalLM.from_pretrained(base_model)
model = PeftModel.from_pretrained(model, adapter_model)
tokenizer = AutoTokenizer.from_pretrained(base_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCa

In [37]:
from tqdm.auto import tqdm

In [38]:
for i in tqdm(range(len(test_set))):
  item = test_set[i]
  result = text_generator(item['prompt'], max_new_tokens=200, return_full_text=False)
  item['response'] = result[0]['generated_text']

  0%|          | 0/196 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin

In [39]:
answers = []
for item in test_set:
  answers.append(item['response'])

In [40]:
print(len(answers))

196


# NDCG & Recall

In [41]:
import re

def parse_recommendations(answer):
  pattern = r'\d+\.\s+"*([^"]+?)"*(?:\s)?(?:\s*-\s*.+)?(?:\s*)?$'
  recommendations = re.findall(pattern, answer, re.MULTILINE)
  return recommendations

In [42]:
import numpy as np

def recall_at_k(r, k, m):
    return (np.asarray(r)[:k] != 0).sum() / m

def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.

def idcg_at_k(k):
    return dcg_at_k(np.ones(k), k)

def ndcg_at_k(r, k, max_relevant):
    idcg = idcg_at_k(min(k, max_relevant))
    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

In [43]:
def evaluate_model(answers, users):
    assert len(answers) == len(users)

    mean_ndcg = 0
    mean_recall = 0
    recommendations_count = 0
    for i in range(len(answers)):
       recommendations = parse_recommendations(answers[i])
       if len(recommendations) != 10:
         continue
       user_data = read_user_data(path_movie_data, users[i]["user_id"])
       user_might_like = [item_map[id] for id in user_data["user_might_like"]]
       print(user_might_like)
       print(recommendations)
       rel_vector = np.isin(recommendations, user_might_like, assume_unique=True).astype(int)

       mean_ndcg += ndcg_at_k(rel_vector, 20, len(user_might_like))
       mean_recall += recall_at_k(rel_vector, 20, len(user_might_like))
       recommendations_count += 1

    mean_ndcg /= recommendations_count
    mean_recall /= recommendations_count

    return mean_ndcg, mean_recall

In [44]:
ndcg, recall = evaluate_model(answers, test_set)
print(f"NDCG: {ndcg:.4f}")
print(f"Recall: {recall:.4f}")

['Movies 4 You - Sci Fi Classics (The Man from Planet X / Beyond the Time Barrier / The Time Travelers / The Angry Red Planet)', 'Coyote Ugly', 'Man Apart VHS', 'The Book Thief', 'American Gigolo', 'Internal Affairs VHS', 'Prey', 'The Onion Field VHS', 'Movies 4 You: More Sci-Fi Classics', 'Thief', 'Big Wednesday VHS', 'Fire Down Below', 'Side Effects', 'The Shallows', 'Dragon Blade']
['Life or Something Like It', 'Dragon Blade', "You've Got Mail", 'Kiss Kiss Bang Bang', 'The Big Sick', 'Mr. & Mrs. Smith', 'Pay It Forward', 'Runaway Bride', 'The Proposal', 'True Lies']
['The Wedding Ringer', 'Meet The Spartans', 'Captain America VHS', 'Poker Night', 'Think Like a Man']
['Think Like a Man', 'The Wedding Singer', '10 Things I Hate About You', "My Best Friend's Wedding", 'How to Lose a Guy in 10 Days', "Bridget Jones's Diary", "You've Got Mail", 'Sleepless in Seattle', 'Miss Congeniality', 'Never Been Kissed']
['Stir Of Echoes']
['Remember the Titans', 'Moneyball', 'A Walk to Remember', '